# Step4: XGBOOST 单模型
需要调整的参数在trainToPredict函数里面
**11-07** v2版本的改动：<br>
1. 修改文件编码为utf-8
2. 输出更多调试信息
3. 调整输出文件后缀为%Y%m%d_%H%M%S

TODO:
1. 利用 MinMaxScaler处理predict值，使得predict值在[0,1]之间

Input：
- Step3产出的经过特征工程处理的特征，../feature/features_all_train_test_20181106.csv

Output：
- 经过训练、交叉验证得到的KS值
- 对线上数据进行预测，模型输出在这里 "../model/"
- 预测结果输出在"../answer/"


In [5]:
import datetime
import pandas as pd
import xgboost as xgb
import time

from sklearn.model_selection import *
from sklearn import metrics
from sklearn import preprocessing

# common prefix
TRAIN_PREFIX = "../train/"
TEST_PREFIX = "../A/"
FEATURE_PREFIX = "../feature/"

# File Encoder
FILE_ENCODER = "utf-8"

def calcKS(y_predict, y_positive):
    label = y_positive
    fpr, tpr, thres = metrics.roc_curve(label, y_predict, pos_label = 1)
    return abs(fpr - tpr).max()

def useModelToPredict(model_file_name, feature_file_name):
    all_data= pd.read_csv(feature_file_name,encoding=FILE_ENCODER) 
    predict_set=all_data[(all_data['标签']<0)]
    print("predict_set shape:")
    print(predict_set.shape)
    ## TO DO
    
def createFeatureMap(features, output_file):  
    outfile = open(output_file, 'w')  
    i = 0  
    for feat in features:  
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))  
        i = i + 1  
    outfile.close()
    
def removeDuplicateFeature(all_data):
    
    #all_data.drop('放款时间_x',axis=1, inplace=True)
    #all_data.drop('放款时间_y',axis=1, inplace=True)
    #all_data.drop('放款时间_x.1',axis=1, inplace=True)
    #all_data.drop('放款时间_y.1',axis=1, inplace=True)

    #检测重复特征名称,并删除
    names=all_data.columns.tolist()
    print(len(names))
    print(len(set(names)))
    print("all_data大小：")
    print(all_data.shape)
    myset = set(names)
    for item in myset:
        if names.count(item)>1:
            print(item)
            dataset=dataset.drop(item,axis=1)
    print("新all_data大小：")
    print(all_data.shape)
    return all_data
    

feature_file_name = "../feature/features_all_train_test_20181106.csv"
#def callXGBoost(feature_file_name):
all_data= pd.read_csv(feature_file_name,encoding=FILE_ENCODER) 

#all_data = removeDuplicateFeature(all_data)
predict_set=all_data[(all_data['标签']<0)]
data_set=all_data[(all_data['标签']>=0)]
# FOR TEST
#data_set = data_set.head(10)
#predict_set = predict_set.head(10)
print("data_set shape:")
print(data_set.shape)
print("predict_set shape:")
print(predict_set.shape)
#trainToPredict(data_set, predict_set)

#def trainToPredict(data_set, predict_set):
#  PARAMS TUNING BEGIN
#  需要调整的参数：


TEST_SIZE = 0.2
RANDOM_STATE = 21

NUM_BOOST_ROUND = 5;
EARLY_STOP_ROUNDS = 500;


train_set = data_set
trainY = train_set['标签']
trainX = train_set.drop(['标签','用户标识'],axis=1)
predictX = predict_set.drop(['标签','用户标识'],axis=1)

#fill -9999 to NAs
trainX.fillna(-9999)
predictX.fillna(-9999)

xgb_model = xgb.XGBClassifier()
#brute force scan for all parameters, here are the tricks
#usually max_depth is 6,7,8
#learning rate is around 0.05, but small changes may make big diff
#tuning min_child_weight subsample colsample_bytree can have 
#much fun of fighting against overfit 
#n_estimators is how many round of boosting
#finally, ensemble xgboost with multiple seeds may reduce variance
parameters = {'nthread':[24], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.01], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [1,2,3],
              'silent': [0],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [NUM_BOOST_ROUND], #number of trees, change it to 1000 for better results
              'missing':[-9999],
              'seed': [1337]}



data_set shape:
(38157, 3108)
predict_set shape:
(5560, 3108)


In [9]:
all_data.shape

(43717, 3108)

In [9]:
print("begin to train a model by XGBoost")
time_now = datetime.datetime.now()
begin_time = time.time()
filename = datetime.datetime.strftime(time_now,'%Y%m%d_%H%M%S')

skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 1001)
grid = GridSearchCV(xgb_model, parameters, n_jobs=1,
               cv=skf.split(trainX, trainY), 
               scoring='roc_auc',
               verbose=3, refit=True, return_train_score=True)

grid.fit(trainX, trainY)

#trust your CV!
print('\n All results:')
print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_)
print('\n Best parameters:')
print(grid.best_params_)
print('\n Best cv_results_:')
results = pd.DataFrame(grid.cv_results_)
results.to_csv('../model/xgb-grid-search-results-01.csv', index=False)

predictY = grid.best_estimator_.predict_proba(predictX)[:,1]

print("XGBoost training finished")
calc_time = time.time()-begin_time
print("cost time: ",calc_time," s")


output = predict_set[['用户标识']]
#output["预测"] = MinMaxScaler().fit_transform(predictY)
output["预测"] = predictY
print("min predict:")
print(min(output["预测"]))
print("max predict:")
print(max(output["预测"]))
output_file=output[['用户标识','预测']].rename(index=str, columns={"用户标识": "userid", "预测": "probability"})



begin to train a model by XGBoost
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=11, missing=-9999, n_estimators=5, nthread=24, objective=binary:logistic, seed=1337, silent=0, subsample=0.7 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[11:52:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[11:52:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=6
[11:52:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=6
[11:52:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[11:52:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=6
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=11, missing=-9999, n_estimators=5, nthread=24, objective=binary:logistic, seed=1337, silent=0, subsample=0.7, score=0.7669037699264911, total=   8.9s
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=11, missing=-9999, n_estimators=5, nthread=24, objective=binary:logistic, seed=1337, silent=0, subsample=0.7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.9s remaining:    0.0s


[11:52:45] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=6
[11:52:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=6
[11:52:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[11:52:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[11:52:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=6
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=11, missing=-9999, n_estimators=5, nthread=24, objective=binary:logistic, seed=1337, silent=0, subsample=0.7, score=0.7867031704443653, total=   8.8s
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=11, missing=-9999, n_estimators=5, nthread=24, objective=binary:logistic, seed=1337, silent=0, subsample=0.7 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   23.8s remaining:    0.0s


[11:52:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[11:52:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=6
[11:52:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=6
[11:53:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=6
[11:53:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=6
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=11, missing=-9999, n_estimators=5, nthread=24, objective=binary:logistic, seed=1337, silent=0, subsample=0.7, score=0.7676792003632481, total=   8.8s
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=11, missing=-9999, n_estimators=5, nthread=24, objective=binary:logistic, seed=1337, silent=0, subsample=0.7 
[11:53:09] src/tree/updat

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.0min finished


[11:53:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[11:53:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 0 pruned nodes, max_depth=6
[11:53:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=6
[11:53:37] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=6
[11:53:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 0 pruned nodes, max_depth=6

 All results:
{'mean_fit_time': array([8.20150833]), 'std_fit_time': array([0.20357474]), 'mean_score_time': array([0.81646428]), 'std_score_time': array([0.03932844]), 'param_colsample_bytree': masked_array(data=[0.7],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_learning_rate': masked_array(data=[0.05],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_max_depth

/data/soft/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [2]:
#Step 4.1 执行XGBoost训练
output_file = callXGBoost("../feature/features_all_train_test_20181106.csv")


3108
3108
all_data大小：
(43717, 3108)
新all_data大小：
(43717, 3108)
data_set shape:
(5, 3108)
predict_set shape:
(5, 3108)
begin to train a model by XGBoost
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    1.3s remaining:    2.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    1.7s remaining:    0.0s


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.